
Open the notebook in Colab https://github.com/gbarbalinardo/ballistico/blob/master/docs/amorphous_presentation.ipynb

<a href="https://github.com/gbarbalinardo/ballistico/blob/master/docs/amorphous_presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Thermal Transport Simulation for Silicon Amorphous
Run the code cell below to install ballistico. Replace username and password.

In [0]:
!pip install git+https://<USERNAME>:<PASSWORD>@github.com/gbarbalinardo/ballistico.git

## Remote Fetch Necessary Files
Run the code cell below to fetch ballistico/finitedifference repos,geometry (in xyz) and forefield files.

In [0]:
import os
if not os.path.exists('dlpoly-216.zip'):
  !wget http://chemac196.ucdavis.edu/data-html/dlpoly-216.tar.gz
  !tar -xzf dlpoly-216.tar.gz
  !rm dlpoly-216.tar.gz



## Import the needed dependencies

In [0]:
from ballistico.finitedifference import FiniteDifference
from ballistico.phonons import Phonons
import matplotlib.pyplot as plt
import numpy as np
import ase.units as units
plt.style.use('seaborn-poster')

# if you want to run it as script
# import sys
# temperature = float(sys.argv[1])
# is_classic = int(sys.argv[2])

## Create the `FiniteDifference` object importing it from the Davide/Charlie format


In [0]:
finite_difference = FiniteDifference.import_from_dlpoly_folder('dlpoly-216')

## Create the `Phonons` object

In [0]:

# # Create a phonon object
phonons = Phonons (finite_difference=finite_difference,
                   is_classic=1,
                   temperature=300,
                   folder='si-amorphous',
                   third_bandwidth=0.05/4.135,
                   broadening_shape='triangle')

## Plot the inverse of the lifetime, $\Gamma$

In [0]:

frequencies = phonons.frequencies.flatten()
gamma_classic = phonons.gamma.flatten()
plt.plot(frequencies[3:], gamma_classic[3:], 'b.', markersize=10)
plt.ylabel('$\Gamma$ (THz)', fontsize=25, fontweight='bold')
plt.xlabel("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylim([gamma_classic.min(), 6])
plt.show()


## Plot phase space

In [0]:

ps_classic = phonons.ps.flatten()
plt.plot(frequencies[3:], ps_classic[3:], 'b.', markersize=10)
plt.ylabel('$\Gamma$ (THz)', fontsize=25, fontweight='bold')
plt.xlabel("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.show()

## Conductivity Quasi-Harmonic-Green-Kubo

In [0]:

print('QHGK conductivity')
cond = phonons.conductivity(method='qhgk').sum(axis=0)

In [0]:
# Create a finite difference object
finite_difference = FiniteDifference.import_from_dlpoly_folder('dlpoly-216')
# '
# # Create a phonon object
temperatures = [5, 10, 15, 20, 35, 50, 75, 100, 150, 200, 300, 500, 1000]
for third_bandwidth in [0.5, 1, 2, 4]:
    conds = []
    for temperature in temperatures:
        phonons = Phonons (finite_difference=finite_difference,
                   is_classic=0,
                   temperature=temperature,
                   folder='si-amorphous',
                   third_bandwidth=third_bandwidth/4.135,
                   broadening_shape='triangle')


        print('AF conductivity')
        cond = (phonons.conductivity(method='qhgk', third_bandwidth=third_bandwidth/4.135).sum(axis=0))
        conds.append(cond.diagonal().mean())
    plt.plot(temperatures, conds, label='$\delta_{in}$='+str(np.round(third_bandwidth/4.135,2))+' THz')
    
plt.ylabel('$\kappa$ (W/m/K)', fontsize=25, fontweight='bold')
plt.xlabel("$Temperature$ (K)", fontsize=25, fontweight='bold')
plt.legend(prop={'size': 20})
plt.show()
